# ReAct

In [7]:
import datetime

from dotenv import load_dotenv
from langchain_community.chat_models import ChatZhipuAI

load_dotenv()

llm = ChatZhipuAI(model="GLM-4-Plus")

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import GoogleSerperResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper

# https://python.langchain.com/docs/integrations/tools/

# TAVILY_API_KEY
# https://api.tavily.com
tavily_search = TavilySearchResults(
    api_wrapper=TavilySearchAPIWrapper(), max_results=3)

# SERPER_API_KEY
# https://serper.dev/api-key
serper_search = GoogleSerperResults(
    api_wrapper=GoogleSerperAPIWrapper(), max_results=3)

# https://duckduckgo.com/ 梯子
# duck_duck_go_search = DuckDuckGoSearchResults(max_results=3, output_format="json")

# https://github.com/searxng/searxng
# https://github.com/searxng/searxng-docker
# searx_search = SearxSearchResults(max_results=3)

query = "梅西最近效力的俱乐部"

tavily_result = tavily_search.invoke({"query": query})
print("tavily_result", tavily_result)
serper_result = serper_search.invoke({"query": query})
print("serper_result", serper_result)

tavily_result [{'url': 'https://www.jiemian.com/article/9521407.html', 'content': '半年前，当自由球员C罗在欧洲难觅下家时，他无奈选择离开；如今，站在同样的十字路口前，无法回到巴萨的梅西也选择了离开。不过，他们一个向左，一个向右，再也难有交集。\n评论\n评论\n下载界面新闻\n微信公众号\n上海界面财联社科技股份有限公司 版权所有\n© 2014-2023 JIEMIAN.COM\n【深度】“梅罗之争”20年走向终章：梅西加盟迈阿密国际，五大联赛绝代双骄统治告终\n随着梅西即将远渡北美，而C罗早已飞抵沙特，绝代双骄的时代彻底落幕。\n杨弋YY · 2023/06/08 08:20来源：界面新闻\n图片来源：视觉中国\n界面新闻记者 | 杨弋\n界面新闻编辑 | 一白\n北京时间2023年6月8日凌晨，阿根廷球星梅西与美国足球大联盟（MLS）迈阿密国际俱乐部相继宣布，双方达成加盟协议，梅西将身穿10号球衣。\n据了解，迈阿密国际为梅西开出5000万欧元年薪的报价，并提供俱乐部股份、Apple以及阿迪达斯赞助协议的丰厚回报。\n梅西表示：“我真的很想回到巴萨，但在两年前发生那样的事情之后，我不想再将自己的未来交于别人之手。考虑到我和我的家人，我决定做出属于自己的选择。”\n至此，持续了近一个赛季的“梅西去哪儿”迎来收官，而职业生涯第二次成为自由球员的梅西，也在短暂“失业”后选定了去处。\n梅西结束了效力欧洲俱乐部19年的时光，他在853场比赛中，贡献704进球338助攻，并斩获38个冠军，7次金球奖。\n如今梅西即将远渡北美，而C罗早在半年前就已飞抵沙特，“绝代双骄”撒手欧洲五大联赛的王权，从此两颗巨星也彻底不再同轨。\n业内看来，从莫德里奇在2018年捧起金球奖起，“梅罗时代”对足坛在竞技领域的统治就已经结束。\n此前十年间，这项象征足坛最高个人荣誉的奖项只在两人间交替易主。这期间，梅西与C罗多次正面较量，各自收获冠军无数，各自登峰造极。世界足球进入了“梅罗时代”。\n2007-2008赛季欧冠半决赛曼联对阵巴萨，开启了“梅罗之争”的序章。23岁的C罗和20岁的梅西第一次同场竞技，最终曼联两回合1-0击败巴萨，两人都没能取得进球。\n跨过巴萨后，曼联在莫斯科的雨夜夺得欧冠冠军。这也成为

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent

search_tools = [tavily_search, serper_search]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Current time: {time}"),
    MessagesPlaceholder("messages")
]).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)

agent_executor = create_react_agent(
    model=llm,
    tools=search_tools,
    state_modifier=prompt_template
)

response = agent_executor.invoke(
    {"messages": [("user", query)]}
)

response.messages[-1].content